<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
    <font color='red'>To begin: Click anywhere in this cell and press <kbd>Run</kbd> on the menu bar. This executes the current cell and then highlights the next cell. There are two types of cells. A <i>text cell</i> and a <i>code cell</i>. When you <kbd>Run</kbd> a text cell (<i>we are in a text cell now</i>), you advance to the next cell without executing any code. When you <kbd>Run</kbd> a code cell (<i>identified by <span style="font-family: courier; color:black; background-color:white;">In[ ]:</span> to the left of the cell</i>) you advance to the next cell after executing all the Python code within that cell. Any visual results produced by the code (text/figures) are reported directly below that cell. Press <kbd>Run</kbd> again. Repeat this process until the end of the notebook. <b>NOTE:</b> All the cells in this notebook can be automatically excecuted sequentially by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart and Run All</kbd>. Should anything crash then restart the Jupyter Kernal by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart</kbd>, and start again from the top.
        
</div>

### 1. Import Modules

In [ ]:
import numpy as np
import pandas as pd
import cimcb as cb

print('All packages successfully loaded')

### 2. Load data and peak sheet

In [ ]:
home = '' 
file = 'ST001047.xlsx' 

DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak') 

### 3. Extract X & Y

In [ ]:
# Clean PeakTable
RSD = PeakTable['QC_RSD']   
PercMiss = PeakTable['Perc_missing']  
PeakTableClean = PeakTable[(RSD < 20) & (PercMiss < 10)]   

# Select subset of Data for the PLS-DA model
DataTable2 = DataTable[(DataTable.Class == "GC") | (DataTable.Class == "HE")]

# Create a Binary Y vector for stratifiying the samples
Outcomes = DataTable2['Class']                                  
Y = [1 if outcome == 'GC' else 0 for outcome in Outcomes]         
Y = np.array(Y)   

# Extract and scale the metabolite data from the DataTable
peaklist = PeakTable['Name']                           
XT = DataTable2[peaklist]                                    
XTlog = np.log(XT)                                          
XTscale = cb.utils.scale(XTlog, method='auto')              
XTknn = cb.utils.knnimpute(XTscale, k=3)   

### 4. Hyperparameters optimisation

In [ ]:
# param_dict
lr = [0.0001,0.001,0.01,0.1,1] 
neurons = [2, 3, 4, 5, 6, 7, 8]
param_dict = dict(learning_rate=lr, n_neurons=neurons, momentum=0.5, decay=0, epochs=400, loss='binary_crossentropy')

# Initalise
cv = cb.cross_val.kfold(model=cb.model.NN_LogitLogit,                      
                                X=XTknn,                                 
                                Y=Y,                               
                                param_dict=param_dict,                   
                                folds=5,
                                n_mc=5)                                

# Run and plot
cv.run()  


In [ ]:
cv.plot(metric='auc', ci=95, color_beta = [5,5,5])

In [ ]:
lr = [0.001,0.005,0.01,0.05,0.1,1]
e = [100,200,400,600,800,1000] 
neurons = [2, 3, 4, 5, 6, 7, 8]
m = [0.2,0.4,0.6,0.8]
param_dict = dict(learning_rate=lr, n_neurons=2, momentum=0.5, decay=0, epochs=e, loss='binary_crossentropy')

# Initalise
cv = cb.cross_val.kfold(model=cb.model.NN_LogitLogit,                      
                                X=XTknn,                                 
                                Y=Y,                               
                                param_dict=param_dict,                   
                                folds=5,
                                n_mc=5)                                

# Run and plot
cv.run()  

In [ ]:
cv.plot(metric='auc', ci=95, color_beta = [5,5,5])

In [ ]:
cv.plot(metric='r2q2', ci=95, color_beta = [5,5,5])

### 6. Get Model and Evaluate

In [ ]:
# Train model
model = cb.model.NN_LogitLogit(learning_rate=0.01, n_neurons=2, momentum=0.5, decay=0, epochs=1000, loss='binary_crossentropy')
model.train(XTknn, Y)
model.test(XTknn)

# Evaluate 
model.evaluate(cutoffscore=0.5, plot_median=False) 

In [ ]:
# Bootstrap evaluate

model.booteval(XTknn, Y, bootnum=10, errorbar=False)

### 7. Save table

In [ ]:
home = ''
file = 'ANNLogitLogit_GastricCancer.xlsx'

model.save_table(home + file)